In [1]:
import json
import re
from pathlib import Path

import pandas as pd
from archetypal.schedule import Schedule

from epinterface.climate_studio.interface import (
    ClimateStudioLibraryV2,
    GlazingConstructionSimple,
    OpaqueConstruction,
    OpaqueMaterial,
    ZoneEnvelope,
    ZoneUse,
)

base_path = Path("../epinterface/data/20241007 Templates update")

with open(base_path / "241007_opaque_constructions.json") as f:
    oc_data = json.load(f)

with open(base_path / "241007_opaque_materials.json") as f:
    om_data = json.load(f)

with open(base_path / "241007_glazing_construction.json") as f:
    gc_data = json.load(f)

with open(base_path / "241007_envelope_construction.json") as f:
    ze_data = json.load(f)

with open(base_path / "241007_space_use.json") as f:
    zu_str = "[" + f.read().replace("\\r\\n", "").replace('\\"', '"')[1:-1] + "]"
with open(base_path / "241007_space_use_v.json", "w") as f:
    f.write(zu_str)
with open(base_path / "241007_space_use_v.json") as f:
    zu_data = json.load(f)

with open(base_path / "241007_year_schedules.json") as f:
    ys_data = json.load(f)


zes: dict[str, ZoneEnvelope] = {}
for val in ze_data:
    val.update({"WWR": 0.15})

    ze = ZoneEnvelope(**val)
    # right an regex to replace "SFH" with "SF" and "MFH" with "MF"
    ze.Name = re.sub(r"(SF|MF)H", r"\1", ze.Name)
    zes[ze.Name] = ze

zus: dict[str, ZoneUse] = {}
for val in zu_data:
    zu = ZoneUse(**val)
    print(zu.Name)
    zus[zu.Name] = zu

oms: dict[str, OpaqueMaterial] = {}
for val in om_data:
    om = OpaqueMaterial(**val)
    oms[om.Name] = om

gcs: dict[str, GlazingConstructionSimple] = {}
for val in gc_data:
    gc = GlazingConstructionSimple(**val)
    gcs[gc.Name] = gc

ocs: dict[str, OpaqueConstruction] = {}
for val in oc_data:
    oc = OpaqueConstruction(**val)
    ocs[oc.Name] = oc

ys: dict[str, Schedule] = {}
df = pd.DataFrame(ys_data[5:29] * 365)
for col in df.columns:
    vals = df[col].tolist()
    sch = Schedule.from_values(col, Values=vals, Type="Fraction")
    ys[col] = sch

lib = ClimateStudioLibraryV2(
    OpaqueConstructions=ocs,
    OpaqueMaterials=oms,
    GlazingConstructions=gcs,
    Envelopes=zes,
    SpaceUses=zus,
    Schedules=ys,
)

MA_SF_pre_1975
MA_SF_btw_1975_2003
MA_SF_post_2003
MA_MF_pre_1975
MA_MF_btw_1975_2003
MA_MF_post_2003


In [2]:
all_series = []
for ze in lib.Envelopes.values():
    facade_missing = ze.Constructions.FacadeConstruction not in lib.OpaqueConstructions
    roof_missing = ze.Constructions.RoofConstruction not in lib.OpaqueConstructions
    ground_slab_missing = (
        ze.Constructions.GroundSlabConstruction not in lib.OpaqueConstructions
    )
    ground_wall_issing = (
        ze.Constructions.GroundWallConstruction not in lib.OpaqueConstructions
    )
    slab_missing = ze.Constructions.SlabConstruction not in lib.OpaqueConstructions
    internal_wall_missing = (
        ze.Constructions.PartitionConstruction not in lib.OpaqueConstructions
    )

    series = pd.Series({
        "name": ze.Name,
        "infiltration": ze.Infiltration.InfiltrationAch
        if ze.Infiltration.CalculationMethod == "AirChanges/Hour"
        else ze.Infiltration.InfiltrationFlowPerExteriorSurfaceArea,
        "facade": ze.Constructions.FacadeConstruction,
        "facade_missing": facade_missing,
        "roof": ze.Constructions.RoofConstruction,
        "roof_missing": roof_missing,
        "ground wall": ze.Constructions.GroundWallConstruction,
        "ground_wall_missing": ground_wall_issing,
        "ground slab": ze.Constructions.GroundSlabConstruction,
        "ground_slab_missing": ground_slab_missing,
        "floor/ceiling": ze.Constructions.SlabConstruction,
        "floor/ceiling missing": slab_missing,
        "internal wall": ze.Constructions.PartitionConstruction,
        "internal_wall_missing": internal_wall_missing,
    })
    all_series.append(series)
df = pd.DataFrame(all_series).set_index("name")
assert df.any(axis=0)[[col for col in df.columns if "missing" in col]].any() == False
print("Envelope Constructions:")
df[
    [
        "facade",
        "roof",
        "floor/ceiling",
        "internal wall",
        "ground slab",
        "ground wall",
        "infiltration",
    ]
]

Envelope Constructions:


,facade,roof,floor/ceiling,internal wall,ground slab,ground wall,infiltration
name,,,,,,,
MA_SF_pre_1975,TEMPLATE_MA_SF_WOOD,MA_SFH_pre_1975_roof,MidriseApartment-5A_INT-FLOOR-TOPSIDE,MidriseApartment-5A_INT-WALLS,90.1-2019 Residential CZ 5 Ground Slab (Mass),MidriseApartment-5A_EXT-SLAB,0.4
MA_SF_btw_1975_2003,MA_SFH_mid_facade,SFH_MA_mid_roof,MidriseApartment-5A_INT-FLOOR-TOPSIDE,MidriseApartment-5A_INT-WALLS,MidriseApartment-5A_EXT-SLAB,MidriseApartment-5A_EXT-SLAB,0.3
MA_SF_post_2003,MA_SFH_post_2003_facade,MA_SFH_new_roof,MidriseApartment-5A_INT-FLOOR-TOPSIDE,MidriseApartment-5A_INT-WALLS,MidriseApartment-5A_EXT-SLAB,MidriseApartment-5A_EXT-SLAB,0.1
MA_MF_pre_1975,TEMPLATE_MF_MA_LOW_facade,MA_MFH_pre_1975_roof,MidriseApartment-5A_INT-FLOOR-TOPSIDE,MidriseApartment-5A_INT-WALLS,MidriseApartment-5A_EXT-SLAB,MidriseApartment-5A_EXT-SLAB,0.8
MA_MF_btw_1975_2003,MidriseApartment-5A_STEEL FRAME RES EXT WALL,MA_MFH_roof_mid_to_new,MidriseApartment-5A_INT-FLOOR-TOPSIDE,MidriseApartment-5A_INT-WALLS,MidriseApartment-5A_EXT-SLAB,MidriseApartment-5A_EXT-SLAB,0.4
MA_MF_post_2003,MidriseApartment-5A_STEEL FRAME RES EXT WALL,MA_MFH_roof_mid_to_new,MidriseApartment-5A_INT-FLOOR-TOPSIDE,MidriseApartment-5A_INT-WALLS,MidriseApartment-5A_EXT-SLAB,MidriseApartment-5A_EXT-SLAB,0.1


In [3]:
print("-----FACADES-----")
for facade_name in df.facade.unique():
    facade = lib.OpaqueConstructions[facade_name]
    print(facade.Name)
    for layer in facade.Layers:
        print(f"\t{layer}")
    print("\n")

print("\n-----ROOFS-----")
for roof_name in df.roof.unique():
    roof = lib.OpaqueConstructions[roof_name]
    print(roof.Name)
    for layer in roof.Layers:
        print(f"\t{layer}")
    print("\n")

print("\n-----GROUND SLABS-----")
for ground_slab_name in df["ground slab"].unique():
    ground_slab = lib.OpaqueConstructions[ground_slab_name]
    print(ground_slab.Name)
    for layer in ground_slab.Layers:
        print(f"\t{layer}")
    print("\n")

print("\n-----GROUND WALLS-----")
for ground_wall_name in df["ground wall"].unique():
    ground_wall = lib.OpaqueConstructions[ground_wall_name]
    print(ground_wall.Name)
    for layer in ground_wall.Layers:
        print(f"\t{layer}")
    print("\n")

print("\n-----FLOOR/CEILINGS-----")
for slab_name in df["floor/ceiling"].unique():
    slab = lib.OpaqueConstructions[slab_name]
    print(slab.Name)
    for layer in slab.Layers:
        print(f"\t{layer}")
    print("\n")

print("\n-----INTERNAL WALLS-----")
for internal_wall_name in df["internal wall"].unique():
    internal_wall = lib.OpaqueConstructions[internal_wall_name]
    print(internal_wall.Name)
    for layer in internal_wall.Layers:
        print(f"\t{layer}")
    print("\n")

-----FACADES-----
TEMPLATE_MA_SF_WOOD
	Name='Stucco' Thickness=0.0157
	Name='Medium density fiberboard' Thickness=0.0157
	Name='Wood wool board 15mm' Thickness=0.089
	Name='Gypsum Board' Thickness=0.0157


MA_SFH_mid_facade
	Name='Stucco' Thickness=0.025
	Name='Concrete' Thickness=0.203
	Name='Mineral Wool' Thickness=0.06331905318300173
	Name='Plaster' Thickness=0.013


MA_SFH_post_2003_facade
	Name='Stucco' Thickness=0.025
	Name='Concrete' Thickness=0.203
	Name='Mineral Wool' Thickness=0.08414746913790223
	Name='Plaster' Thickness=0.013


TEMPLATE_MF_MA_LOW_facade
	Name='Stucco' Thickness=0.003
	Name='Concrete' Thickness=0.03
	Name='Oriented strand board' Thickness=0.011
	Name='Foam glass' Thickness=0.1397
	Name='Plaster' Thickness=0.0157


MidriseApartment-5A_STEEL FRAME RES EXT WALL
	Name='MidriseApartment-5A_WOOD SIDING' Thickness=0.01
	Name='MidriseApartment-5A_STEEL FRAME RES WALL INSULATION' Thickness=0.119161363
	Name='MidriseApartment-5A_1/2IN GYPSUM' Thickness=0.0127



-----

In [4]:
use = [k for k in list(ocs.keys()) if "sf" in k.lower() or "mf" in k.lower()]
for k in use:
    print(k)
# print("--")
# for k in list(ocs.keys()):
#     if k not in use:
#         print(k)

Template_MA_SF_OLD
Template_MA_MF_NEW_roof
TEMPLATE_MF_MA_LOW_facade
TEMPLATE_MF_MA_HIGH_facade
Template_MA_SF_OLD_roof
SFH_MA_mid_roof
MA_SFH_new_roof
TEMPLATE_MA_MFH_WOOD
SFH_ext_floor_ALL
SFH_roof_post_2003
SFH_ground_slab
SFH_ground_wall
TEMPLATE_MA_SF_WOOD
MA_SFH_post_2003_facade
MA_SFH_pre_1975_roof
MA_MFH_pre_1975_roof
MA_SFH_mid_facade
MA_MFH_roof_mid_to_new


In [5]:
for oc in ocs.values():
    for mat in oc.Layers:
        if mat.Name not in oms:
            print(f"Missing material {mat.Name} in construction {oc.Name}")

In [6]:
all_data = []
for su in lib.SpaceUses.values():
    name = su.Name
    data = su.Loads.model_dump()
    data.update(su.Conditioning.model_dump())
    data.update({"Name": name})
    all_data.append(data)

df = pd.DataFrame(all_data).set_index("Name")

df[
    [
        "EquipmentPowerDensity",
        "LightingPowerDensity",
        "PeopleDensity",
        "OccupancySchedule",
        "LightsAvailabilitySchedule",
        "EquipmentAvailabilitySchedule",
        "MechVentIsOn",
        "HeatingSetpoint",
        "CoolingSetpoint",
        # "MaxHeatSupplyAirTemp",
        # "MinCoolSupplyAirTemp",
        # "Autosize",
        # "HumidistatIsOn"
        # "CoolingSetpointConstant",
        # "HeatingSetpointConstant",
        # "HeatingLimitType",
        # "CoolingLimitType"
    ]
]
# df.columns

,EquipmentPowerDensity,LightingPowerDensity,PeopleDensity,OccupancySchedule,LightsAvailabilitySchedule,EquipmentAvailabilitySchedule,MechVentIsOn,HeatingSetpoint,CoolingSetpoint
Name,,,,,,,,,
MA_SF_pre_1975,3.4,2.5,0.0220,MA_OCC_SCH,MA_LIGHTS_SCHEDULE,MA_EQUIP_SCHEDULE,False,19.0,26.0
MA_SF_btw_1975_2003,2.7,2.4,0.0219,MA_OCC_SCH,MA_LIGHTS_SCHEDULE,MA_EQUIP_SCHEDULE,False,20.0,24.0
MA_SF_post_2003,2.5,2.1,0.0160,MA_OCC_SCH,MA_LIGHTS_SCHEDULE,MA_EQUIP_SCHEDULE,False,20.0,24.0
MA_MF_pre_1975,3.8,3.5,0.0250,MA_OCC_SCH,MA_LIGHTS_SCHEDULE,MA_EQUIP_SCHEDULE,False,21.0,23.0
MA_MF_btw_1975_2003,2.6,3.2,0.0210,MA_OCC_SCH,MA_LIGHTS_SCHEDULE,MA_EQUIP_SCHEDULE,False,21.0,23.0
MA_MF_post_2003,2.2,2.6,0.0220,MA_OCC_SCH,MA_LIGHTS_SCHEDULE,MA_EQUIP_SCHEDULE,False,21.0,23.0


In [7]:
all_windows = []
for window in lib.GlazingConstructions.values():
    all_windows.append(window.model_dump())

df = pd.DataFrame(all_windows).set_index("Name")
df[["SHGF", "UValue", "TVis"]]
# df

,SHGF,UValue,TVis
Name,,,
Template_pre_1975,0.39,3.804260,0.898
Template_btw_1975_2003,0.40,2.839132,0.440
Template_post_2003,0.38,2.044175,0.418


In [8]:
for zu in zus.values():
    zu.Loads.LightingPowerDensity *= 1
    zu.Loads.EquipmentPowerDensity *= 1
    zu.HotWater.FlowRatePerPerson = 0.00167
    zu.HotWater.WaterSupplyTemperature = 60
    zu.HotWater.WaterTemperatureInlet = 10
    zu.HotWater.DomHotWaterCOP = 1.0
    zu.HotWater.HotWaterFuelType = "NaturalGas"
    zu.HotWater.IsOn = True
    zu.HotWater.WaterSchedule = "AllOn"

In [9]:
with open("everett_lib.json", "w") as f:
    json.dump(lib.model_dump(mode="json"), f, indent=2)

In [9]:
pd.read_csv(base_path / "expected_template_values.csv").drop(
    columns=["Assumed_COP", "Unit"]
).set_index(["Template", "Energy_type"]).drop("Total", level="Energy_type").unstack(
    "Energy_type"
).loc[
    [
        "MA_SFH_pre_1975",
        "MA_SFH_btw_1975_2003",
        "MA_SFH_post_2003",
        "MA_MFH_pre_1975",
        "MA_MFH_btw_1975_2003",
        "MA_MF_post_2003",
    ]
]

Real_value_exp          CS_value_exp        
Energy_type                 Heating  Cooling      Heating Cooling
Template                                                         
MA_SFH_pre_1975             100-150    30-45       103.04   35.16
MA_SFH_btw_1975_2003         75-120    20-32       101.85   23.23
MA_SFH_post_2003             55-100  6 to 23        73.19    9.19
MA_MFH_pre_1975             100-168    25-45       158.90   40.39
MA_MFH_btw_1975_2003         75-148    20-41       120.47   37.58
MA_MF_post_2003              25-100    15-30        93.25   19.23

In [10]:
gt = "GROUND TEMPERATURES,3,.5,,,,1.11,0.10,1.89,4.69,12.02,17.68,21.50,22.66,20.68,16.29,10.42,4.97,2,,,,4.82,3.02,3.39,4.88,9.79,14.22,17.77,19.67,19.26,16.77,12.73,8.44,4,,,,7.90,6.05,5.65,6.21,8.98,11.97,14.71,16.62,17.06,15.98,13.61,10.71"

vals = gt.split(",")
ct = int(vals[1])
temps = [[float(v) for v in vals[6 + i * 16 : 18 + i * 16]] for i in range(ct)]
depths = [float(v) for v in vals[2::16]]
import numpy as np

np.array(temps)
depths
temps[0]

[1.11, 0.1, 1.89, 4.69, 12.02, 17.68, 21.5, 22.66, 20.68, 16.29, 10.42, 4.97]

In [12]:
from epinterface.weather import BaseWeather, WeatherUrl

url = WeatherUrl(
    "https://climate.onebuilding.org/WMO_Region_4_North_and_Central_America/USA_United_States_of_America/MA_Massachusetts/USA_MA_Boston-Logan.Intl.AP.725090_TMYx.2009-2023.zip"
)
base = BaseWeather(Weather=url)
e, d = await base.fetch_weather("cache")